In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import pickle
import os

In [ ]:
import torch 

In [8]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 125)

In [2]:
model_path_name = "nn_single_model"
DIRECTORY_PATH = f"models/{model_path_name}"
MODEL_INFO_FILEPATH = f"{DIRECTORY_PATH}/info.txt"

if not os.path.exists(DIRECTORY_PATH):
    os.makedirs(DIRECTORY_PATH)
with open(MODEL_INFO_FILEPATH, "w") as text_file:
    print(f"##### MODEL INFO #####", file=text_file)

In [3]:
dataset = pd.read_pickle("data/dataset.pkl")

In [4]:
train_index_stop = 569484 # this is the first index of a new pitcher
val_index_stop = 640690 # make sure no overlapping plays

training_set = dataset.iloc[0:train_index_stop, :]
validation_set = dataset.iloc[train_index_stop:val_index_stop, :]
test_set = dataset.iloc[val_index_stop:,]

train_X = training_set.drop(["uid", "pitch_type", "type_confidence"],axis=1)
train_y = training_set['pitch_type']
train_weights = training_set['type_confidence']

val_X = validation_set.drop(["uid", "pitch_type", "type_confidence"],axis=1)
val_y = validation_set['pitch_type']
val_weights = validation_set['type_confidence']

In [5]:
mapping = {'FF' : 0,
           'SL': 1, 
           'CU': 2, 
           'SI': 3, 
           'FC': 4, 
           'FT': 5, 
           'KC': 6, 
           'CH': 7, 
           'KN': 8, 
           'FS': 9, 
           'FO': 10, 
           'EP': 11, 
           'SC': 12}

In [6]:
train_y = train_y.map(mapping)
val_y = val_y.map(mapping)

In [9]:
train_X

,inning,top,score_diff,at_bat_num,p_throws,pcount_pitcher,bases_state,pitcher_id,pcount_at_bat,balls,strikes,outs,stand,height_inches,prev_pitch_type,prev_type,prev_end_speed,prev_break_length,prev_break_angle,prev_break_y,prev_zone,prev_spin_dir,prev_spin_rate
0,1,False,0,7,R,1,000,450308,1,0,0,0,R,70,NaN,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,False,0,7,R,2,000,450308,2,0,1,0,R,70,FF,R,81.4,2.8,-0.7,23.9,8.0,183.148,2519.455
2,1,False,0,7,R,3,000,450308,3,1,1,0,R,70,FF,R,84.0,1.9,6.9,23.8,12.0,187.663,2838.803
3,1,False,0,7,R,4,000,450308,4,2,1,0,R,70,FF,R,83.8,2.3,-12.4,23.9,14.0,179.643,2701.919
4,1,False,0,7,R,5,000,450308,5,2,2,0,R,70,FF,R,84.0,2.1,0.1,23.8,3.0,184.623,2683.280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569479,3,True,0,17,L,41,010,450306,5,1,2,1,L,70,CH,L,72.5,11.2,-24.8,23.7,7.0,85.433,2206.388
569480,3,True,0,18,L,42,001,450306,1,0,0,2,R,71,NaN,R,NaN,NaN,NaN,NaN,NaN,NaN,NaN
569481,3,True,0,18,L,43,001,450306,2,0,1,2,R,71,FF,R,79.9,6.2,-24.3,23.6,2.0,127.290,1773.896
569482,3,True,0,18,L,44,001,450306,3,0,2,2,R,71,CH,R,73.0,10.7,-21.4,23.7,14.0,86.928,1822.046


In [ ]:
# Construct our model by instantiating the class defined above
model = Polynomial3()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters (defined 
# with torch.nn.Parameter) which are members of the model.
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=3e-4)
for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()